In [1]:
import pandas as pd
import requests

In [2]:
# how to scrape

# txt
# from: <div class="user-post__text">
# to: </div>

# score
# from: <span class="user-post__score-count">
# to: </span>

In [3]:
ProductIds = []

for i in range(1000):
    url = f"https://www.ceneo.pl/Sprzet_RTV;0020-30-0-0-{i}.htm"
    r = requests.get(url, allow_redirects=True)
    txt = r.text

    while 1:
        try:
            ProductId_from = txt.find('''data-ProductId="''')+16
            ProductId_till = txt[ProductId_from:].find('''"''')
            if ProductId_from == 15:
                break
            # print(txt[ProductId_from:ProductId_from+ProductId_till])
            ProductIds.append(txt[ProductId_from:ProductId_from+ProductId_till])
            txt = txt[ProductId_from+ProductId_till:]
        except:
            pass

In [4]:
num = 111013752
# num = 100000000
df = pd.DataFrame(columns=['txt','score'])

for pid in ProductIds:
    url = f"https://www.ceneo.pl/{pid}#tab=reviews"
    r = requests.get(url, allow_redirects=True)
    txt = r.text

    while 1:
        score_from = txt.find('''<span class="user-post__score-count">''') + 37
        score_till= txt[score_from:].find('''</span>''')
        curr_score = txt[score_from:score_from+score_till]
        txt = txt[score_from+score_till:]

        review_from = txt.find('''<div class="user-post__text">''') + 29
        review_till = txt[review_from:].find('''</div>''')
        curr_review = txt[review_from:review_from+review_till]

        if score_till > 6:
            break

        txt = txt[review_from+review_till:]

        df = pd.concat([pd.DataFrame(data={'txt':[curr_review],'score':[curr_score]}),df])

        # print(score_from, score_till, review_from, review_till)
        # print(curr_review, curr_score)

ConnectionError: HTTPSConnectionPool(host='www.ceneo.pl', port=443): Max retries exceeded with url: /135227711 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000023863D9DD30>: Failed to establish a new connection: [WinError 10060] Próba połączenia nie powiodła się, ponieważ połączona strona nie odpowiedziała poprawnie po ustalonym okresie czasu lub utworzone połączenie nie powiodło się, ponieważ połączony host nie odpowiedział'))

In [6]:
df.to_excel("new.xlsx")

In [5]:
df

,txt,score
0,Nie polecam. Po pewnym czasie użytkowania zauw...,"0,5/5"
0,Telewizor do oglądania kolorowych treści nadaj...,2/5
0,Jakość obrazu bardzo dobra w trybie dynamiczny...,5/5
0,Jestem zadowolony. Tv że średniej półki. Zapła...,5/5
0,"Obraz super, dźwięk - TRAGEDIA. Głośniczki jak...",3/5
...,...,...
0,Super TV Polecam!!!!!!,5/5
0,jest super jednak po tygodniu trudno ocenić n...,5/5
0,Super telewizor. Bardzo dużo udogodnień w obsł...,5/5
0,"Wszystko super. Jakość obrazu, dźwięku naprawd...",5/5
